<a href="https://colab.research.google.com/github/katemartian/FiberPhotometryDataAnalysis/blob/master/LHA_Vglut2_airpuff2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>LHA-Vglut2-cre Airpuff 2</center></h1>

# Set up workspace

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%run /content/drive/My\ Drive/!lab/FiberPhotometryDataAnalysis.ipynb

All FP functions are ready to use


In [0]:
%cd /content/drive/My Drive/!lab/!FP/2020-02-06 LHA-Vglut2 Airpuff 2/
folder = os.getcwd()
files = find_file(folder+'/raw/','.csv')
files

/content/drive/My Drive/!lab/!FP/2020-02-06 LHA-Vglut2 Airpuff 2


['glut4_LHb_VTA_DRN_airpuff2_0.csv',
 'glut2_LHb_VTA_DRN_airpuff2_10.csv',
 'glut3_LHb_VTA_DRN_airpuff2_12.csv',
 'glut1_LHb_VTA_DRN_airpuff2_8.csv']

# Function to get data


In [0]:
def get_data(name):

 # Experiment info -------------------------------------------------------------
  sepname = name.split('_')  
  # Create a dictionary of Experiment info        
  exp_info = {'mouse': sepname[0],
              'output': ['LHA-'+sepname[1],'LHA-'+sepname[2],'LHA-'+sepname[3]],
              'test': sepname[4]}  


 # Read FP data frame ----------------------------------------------------------
  df = pd.read_csv('./raw/'+name,sep=',',index_col=False)


 # Data specific work around ---------------------------------------------------
 # Remove additional lines for F pressings
  remove = []
  for i in range(len(df)):
    if np.all(df.iloc[i,1:7].isnull().all()):
      if df['F1 Event'][i]==1:
        time = float(df['---'][i])
        prev_t = float(df['---'][i-1])
        next_t = float(df['---'][i+1])
        if abs(time - prev_t) < 0.001:
          df.loc[i-1,'F1 Event'] = 1
        if abs(time - next_t) < 0.001:
          df.loc[i+1,'F1 Event'] = 1
      remove.append(i)
 # Remove additional lines with weird time stamps 
  for i in range(2,len(df)):
    if float(df['---'][i]) - float(df['---'][i-1]) > 1:
      remove.append(i)
 # Remove indices
  df.drop(remove, inplace=True)
  df.reset_index(inplace=True,drop=True)
 # Data frame length should be even to split into reference and signal
  if len(df) % 2 == 0:
    df.drop(df.index[-1],inplace=True)


 # Experiment time -------------------------------------------------------------
  exp_time = np.array(df.iloc[1::2,0].astype(np.float))
  period = find_avg_period(exp_time)
  exp_info['time'] = exp_time
  exp_info['period'] = period
  exp_info['frequency'] = 1/period


 # Create dictionary of raw signal ---------------------------------------------
  Raws = {}
  for fiber,output in enumerate(exp_info['output']):
    Raws[output] = {'reference': np.array(df.iloc[1::2,fiber*2+1].astype(np.float)),
                    'signal': np.array(df.iloc[2::2,fiber*2+2].astype(np.float))}


 # Behavioral events -----------------------------------------------------------
  f1 = np.array(df['F1 Event'].astype(np.float))
  ap = [float(df['---'][i]) for i in range(len(f1)) if f1[i]==1]
 # Create dictionary of vectors of events 
  events = {'airpuff': np.array(ap).reshape(len(ap),1)}


  return exp_info, events, Raws

In [0]:
# Info to create mean plots
info_for_array = {'airpuff': {'window': [-5,5],
                             'avg_win': [-2,-1],
                             'figsize': (12,10)}}

# Make all plots


In [0]:
for name in files:

  print(name)

  exp_info,events,Raws = get_data(name)

  Intensities = allrecordings_preprocess(exp_info,events,Raws,plot=True,save=True)

  Alligned = allrecordings_align(exp_info,events,Intensities,plot=True,save=True)

  dFFs = allrecordings_dff(exp_info,events,Alligned,plot=True,save=True)

  Events_arrays = allrecordings_events_arrays(exp_info,events,dFFs,info_for_array=info_for_array,plot=True,save=True)

  plt.close('all')

print('Done!')

glut4_LHb_VTA_DRN_airpuff2_0.csv
glut2_LHb_VTA_DRN_airpuff2_10.csv
glut3_LHb_VTA_DRN_airpuff2_12.csv
glut1_LHb_VTA_DRN_airpuff2_8.csv
Done!


In [0]:
%debug